<a href="https://colab.research.google.com/github/nestorjimeno/Near-Earth-Objects/blob/master/Ecowitt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Obtención de datos de la API de Ecowitt**

- 5 minutes resolution data within the past 90 days, each data request time span should not be longer than a complete day.
- 30 minutes resolution data within the past 365 days, each data request time span should not be longer than a complete week.
- 240 minutes resolution data within the past 730 days, each data request time span should not be longer than a complete month.
- 24hours resolution data since 2019.1.1, each data request time span should not be longer than a complete year.

Inicializo las constantes que necesito para acceder a la API e importo las librerías que necesito.

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta


APPLICATION_KEY='A943A57F2E8977048B04A7E37C083ECA'
API_KEY='3af577b2-379d-467a-b7ca-19e4592121b6'
MAC='C4:5B:BE:6D:ED:BB'
URL='https://api.ecowitt.net/api/v3/device/history'

Defino la clase Device:

In [ ]:
class Device():
    def __init__(self, mac=MAC):
        self.mac = mac
        self.latitude, self.longitude = self.get_location()

    def get_location(self):
        params = {
            'application_key': APPLICATION_KEY,
            'api_key': API_KEY,
            'mac': self.mac
        }
        try:
            response = requests.get(
                url='https://api.ecowitt.net/api/v3/device/info',
                params=params
            )
            response.raise_for_status()
            data = response.json()
            latitude = data['data']['latitude']
            longitude = data['data']['longitude']
            return latitude, longitude
        except requests.exceptions.RequestException as e:
            print(f"Error en la solicitud: {e}")
            return None, None
        except KeyError as e:
            print(f"Error al procesar la respuesta: {e}")
            return None, None

    def get_data(self,
                start_date,
                end_date,
                call_back='outdoor,indoor,solar_and_uvi,rainfall,wind,pressure',
                cycle_type='5min',
                temp_unitid=1,
                pressure_unitid=5,
                wind_speed_unitid=7,
                rainfall_unitid=12,):

        params = {
            'application_key': APPLICATION_KEY,
            'api_key': API_KEY,
            'mac': self.mac,
            'start_date':self.convert_date(start_date),
            'end_date': self.convert_date(end_date),
            'call_back': call_back ,
            'cycle_type': cycle_type ,
            'temp_unitid': temp_unitid ,
            'pressure_unitid': pressure_unitid ,
            'wind_speed_unitid': wind_speed_unitid ,
            'rainfall_unitid': rainfall_unitid
        }
        try:
            response = requests.get(
                url='https://api.ecowitt.net/api/v3/device/history',
                params=params
            )
            response.raise_for_status()
            data = response.json()
            return data
        except requests.exceptions.RequestException as e:
            print(f"Error en la solicitud: {e}")
            return None
        except KeyError as e:
            print(f"Error al procesar la respuesta: {e}")
            return None


    def convert_date(self, date_str):
        formats = ['%d/%m/%Y %H:%M', '%Y-%m-%d %H:%M', '%d/%m/%Y']
        output_format = '%Y-%m-%d %H:%M'

        for fmt in formats:
            try:
                # Intentar convertir la cadena de entrada a un objeto datetime usando el formato actual
                dt = datetime.strptime(date_str, fmt)
                # Convertir el objeto datetime al formato de salida
                return dt.strftime(output_format)
            except ValueError:
                # Si el formato no coincide, intentar el siguiente
                continue

        # Si ninguno de los formatos coincide, retornar un mensaje de error
        return "Error: No se pudo analizar la fecha con los formatos proporcionados."


Inicializo el dispositivo.

In [ ]:
dev = Device()

## Obtención de los datos de un año

In [ ]:
data_dict = dev.get_data('2024-01-01 00:00', '27/08/2024 23:00')

In [ ]:
data_dict.keys()

dict_keys(['code', 'msg', 'time', 'data'])

In [ ]:
data_dict['data'].keys()

dict_keys(['outdoor', 'indoor', 'solar_and_uvi', 'rainfall', 'wind', 'pressure'])

In [ ]:
o = data_dict['data']['outdoor'].keys()
i = data_dict['data']['indoor'].keys()
s = data_dict['data']['solar_and_uvi'].keys()
r = data_dict['data']['rainfall'].keys()
w = data_dict['data']['wind'].keys()

In [ ]:
print(o)
print(i)
print(s)
print(r)
print(w)

dict_keys(['temperature', 'feels_like', 'app_temp', 'dew_point', 'humidity'])
dict_keys(['temperature', 'humidity'])
dict_keys(['solar', 'uvi'])
dict_keys(['rain_rate', 'daily', 'event', 'hourly', 'weekly', 'monthly', 'yearly'])
dict_keys(['wind_speed', 'wind_gust', 'wind_direction'])


In [ ]:
for item in o:
    print(len(data_dict['data']['outdoor'][item]['list']))
for item in i:
    print(len(data_dict['data']['indoor'][item]['list']))
for item in s:
    print(len(data_dict['data']['solar_and_uvi'][item]['list']))
for item in r:
    print(len(data_dict['data']['rainfall'][item]['list']))
for item in w:
    print(len(data_dict['data']['wind'][item]['list']))

240
240
240
240
240
240
240
240
240
240
240
240
240
240
240
240
240
240
240
